# CS7324 - Lab 3: Extending Logistic Regression
### Jarad Angel, Zach Bohl,  and Luigi Allen

Airline Passenger Satisfation Dataset: https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction

# Preparation and Overview

### Task Explanation and Business Case

In [ ]:
%pip install pandas 
%pip  install matplotlib
%pip install seaborn

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Loading the dataset and displaying feature information for analysis
df = pd.read_csv('Archive/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

In [ ]:
#one hot encoding


In [ ]:
#Dementionality reduction

# Modeling

# Deployment

# Exceptional Work 